In [ ]:
#### load libs

%matplotlib inline

import math, keras, datetime, pandas as pd, numpy as np, keras.backend as K
import matplotlib.pyplot as plt, operator, random, pickle
import os
import xgboost as xgb

from isoweek import Week
from pandas_summary import DataFrameSummary

In [ ]:
path = "F:/AV/AbInBev/"
train_folder = 'train_OwBvO8W'
test_folder = 'test_8uviCCm'
submit_folder = 'sample_submission_1'

In [ ]:
for i in os.walk(path+train_folder):
    print(i)

In [ ]:
### functoin to get csvs within a folder

def get_csv(path,folder):
    for path,directory,files in os.walk(path+folder):
        return(path,directory,[path+"/"+file for file in files])

In [ ]:
### train details
tr_path,_,tr_files=get_csv(path,train_folder)

In [ ]:
#train_tables_csv
train_tables = [pd.read_csv(file,low_memory=False) for file in tr_files]

In [ ]:
from IPython.display import HTML

In [ ]:
### take a look at head of table
for t in train_tables: display(t.head())

In [ ]:
tr_files

In [ ]:
#### lets check the summary for train tables

for t in train_tables: display(DataFrameSummary(t).summary())


In [ ]:
#### no missing values in any field

#### Data Cleaning/Feature engineering

In [ ]:
### separate out into each individual table
demographics,event_cal,hist_volm,ind_soda_sales,ind_volm,promo,weather = train_tables

#### We will create multiple columns indicating which weeks had which kind of holiday/event and a combination of those

In [ ]:
#### lets add more info to event_calendar dataframe based on state holidays, sports, music festivals, beer related stuff
state_hols = ['Easter Day','Good Friday','New Year','Christmas','Labor Day','Independence Day','Revolution Day Memorial']
sports = ['Regional Games ','FIFA U-17 World Cup','Football Gold Cup']

event_cal['National_Hol'] = np.where(np.sum(event_cal[state_hols],axis=1)>0,1,0) 
event_cal['Sports_Week'] = np.where(np.sum(event_cal[sports],axis=1)>0,1,0) 

event_cal['National_Hol_and_Sports_Week'] = np.where(np.sum(event_cal[['National_Hol','Sports_Week']],axis=1)>1,1,0) 
event_cal['Beer_Capital_and_Sports'] = np.where(np.sum(event_cal[['Beer Capital','Sports_Week']],axis=1)>1,1,0) 
event_cal['Beer_Capital_and_Music Fest'] = np.where(np.sum(event_cal[['Beer Capital','Music Fest']],axis=1)>1,1,0) 
event_cal['Sports_Week_and_Music Fest'] = np.where(np.sum(event_cal[['Sports_Week','Music Fest']],axis=1)>1,1,0) 
event_cal['National_Hol_and_Music Fest'] = np.where(np.sum(event_cal[['National_Hol','Music Fest']],axis=1)>1,1,0) 
event_cal.head()

In [ ]:
#### function to retrieve last event for all yearmonths

def prior_event_track(df,event):
    col = []
    last_YearMonth = 201301 #000000
    last_event_val = df.iloc[0,:][event]     
    
    for i in range(len(df)):
        if True:
            curr_val_event = df.iloc[i,:][event]        
            curr_YearMonth = df.iloc[i,:]['YearMonth']
#             print(curr_val_event)
#             print(curr_YearMonth)   
            
        if curr_val_event == 0 :
            mth_begin = str(curr_YearMonth)[4:]
            mth_end   = str(last_YearMonth)[4:]
            yr_begin = str(curr_YearMonth)[:4]
            yr_end   = str(last_YearMonth)[:4]
            
            mth_diff = int(mth_begin) - int(mth_end)
            yr_diff = int(yr_begin) - int(yr_end)
            
            if yr_diff>0:
                mth_diff += yr_diff*12
            if last_event_val == 0:
                col.append(99)
            if last_event_val != 0:
                col.append(mth_diff)
            
        if curr_val_event > 0 :
            last_YearMonth = curr_YearMonth
            last_event_val = curr_val_event
            col.append(0)
    
    df['time_since_'+event] = col        
    
    return (df)

In [ ]:
### Add all prior event info to useful general columns

prior_event_track(event_cal,'National_Hol')
prior_event_track(event_cal,'Sports_Week')
prior_event_track(event_cal,'National_Hol_and_Sports_Week')
prior_event_track(event_cal,'Beer_Capital_and_Sports')
prior_event_track(event_cal,'Beer_Capital_and_Music Fest')
prior_event_track(event_cal,'Music Fest')
event_cal.head()

In [ ]:
#### Add per capita income to demographics table

demographics['Per_Capita'] = demographics['Avg_Yearly_Household_Income_2017']/demographics['Avg_Population_2017']

In [ ]:
### lets also see how each row/Agency's demographcis relate to overall max and min for pop and income
def update_demographics_max_min(dataframe,columns):
    for column in columns:
        feat_max =  dataframe[column].max()
        feat_min =  dataframe[column].min()
        feat_avg =  dataframe[column].mean()
        dataframe[column+"_max_diff"] = feat_max - dataframe[column]
        dataframe[column+"_min_diff"] = feat_min - dataframe[column]
        dataframe[column+"_avg_diff"] = feat_avg - dataframe[column]
        del feat_max,feat_min,feat_avg
    return(dataframe)

In [ ]:
demographics = update_demographics_max_min(demographics,['Avg_Population_2017','Avg_Yearly_Household_Income_2017','Per_Capita'])
demographics.head()

In [ ]:
#### lets try facebook's prophet forecasting lib
import datetime
from fbprophet import Prophet

### we need 2 columns for each agency: namely DS and Y (DS is date and Y is numeric)
weather['YearMonth'] = weather['YearMonth'].astype(str)+('01')
weather['ds'] = pd.to_datetime(weather['YearMonth'].astype(str), format='%Y%m%d')
weather['y'] = weather['Avg_Max_Temp']
weather.head()

In [ ]:
#### predict last data point for max temp for 201801 (takes a min or two)
pred_temp = []
for agency in np.unique(weather['Agency']):
    # agency = 'Agency_60'
    agency = agency
    Avg_Temp = weather.loc[weather['Agency']==agency]
    m= Prophet().fit(Avg_Temp)
    future = m.make_future_dataframe(periods=1,freq = 'M')
    fcst = m.predict(future)
    m.plot(fcst)
    pred = fcst.iloc[-1,-1]
    pred_temp.append({agency:pred})
    del m, Avg_Temp,fcst,future

In [ ]:
#### Build am agency location dataframe
pred_temp_df = pd.DataFrame(np.zeros((len(pred_temp), 2)))
pred_temp_df.columns = ['Agency',"Avg_Max_Temp"]
idx = 0
for i in pred_temp:
    #print(i)
    for k,v in i.items():
        #print(k,v)
        pred_temp_df.iloc[idx,0] = k
        pred_temp_df.iloc[idx,1] = v
    idx += 1

pred_temp_df['YearMonth'] = '20180101'
pred_temp_df = pred_temp_df[['YearMonth','Agency','Avg_Max_Temp']]
pred_temp_df.head()

### Seems like some agencies are in the same location, lets add that info 


In [ ]:
#### merge the 201801 prediction with actual file

weather.drop(['ds','y'],inplace=True,axis=1)
weather = pd.concat([weather,pred_temp_df])
weather['YearMonth'] = weather['YearMonth'].astype(str).str[:-2].astype(np.int64)
weather.head()
### this file has original data and prediction for final month

In [ ]:
#### from https://stackoverflow.com/questions/20672238/find-dictionary-keys-with-duplicate-values

### output: list with unqiue next month temp pred value and the respective agency

flipped = {}
for i in pred_temp:
    for key, value in i.items():
        if value not in flipped:
            flipped[value] = [key]
        else:
            flipped[value].append(key)

In [ ]:
agency_location = []
i = 0
for k,v in flipped.items():
    for item in v:
        agency_location.append({item:"location"+str(i)})
        #print(v)
    i += 1

In [ ]:
#### build an agency location dataframe --- this is only a hypothesis

Agency_Location_DF = pd.DataFrame(np.zeros((len(agency_location), 2)))
Agency_Location_DF.columns = ['Agency',"Location"]
idx = 0
for i in agency_location:
    print(i)
    for k,v in i.items():
        print(k,v)
        Agency_Location_DF.iloc[idx,0] = k
        Agency_Location_DF.iloc[idx,1] = v
    idx += 1

In [ ]:
Agency_Location_DF.head()

In [ ]:
### lets merge the weather and location dataframes
Weather_Location_merge = weather.merge(Agency_Location_DF,how='left',on="Agency")

### We will not look into other files such a soda vol, industry volume but will work on the below:

- Agency_Location_DF
- weather
- demographics
- event_cal
- hist_volm

In [ ]:
hist_volm.head()

#### Baseline --- Not using other files only historical volumnes and facebook package



In [ ]:
hist_volm_upd = hist_volm.copy()

In [ ]:
hist_volm_upd.head()

In [ ]:
### we need 2 columns for each agency: namely DS and Y (DS is date and Y is numeric)
hist_volm_upd['YearMonth'] = hist_volm_upd['YearMonth'].astype(str)+('01')
hist_volm_upd['ds'] = pd.to_datetime(hist_volm_upd['YearMonth'].astype(str), format='%Y%m%d')
hist_volm_upd['y'] = hist_volm_upd['Volume']
hist_volm_upd.head()

In [ ]:
hist_volm_upd['Agency_SKU'] = hist_volm_upd['Agency']+"-"+hist_volm_upd['SKU']
hist_volm_upd.head()

In [ ]:
#### predict last data point for each agency - SKU combination
from tqdm import tqdm
import gc
import warnings
warnings.filterwarnings("ignore")

pred_volm = []

for agency_sku in tqdm(np.unique(hist_volm_upd['Agency_SKU'])):
    agency,sku = agency_sku.split("-")
    pred = 0
    rt = 0
    sku = sku
    agency = agency

    Volm_df = hist_volm_upd.loc[(hist_volm_upd['Agency']==agency) & (hist_volm_upd['SKU']==sku)]

    if len(Volm_df) == 0:
        pred = 0
        pred_volm.append({agency+"-"+sku:int(pred)})
        rt +=1
        #print(rt)

    elif ((len(Volm_df.loc[Volm_df['YearMonth']=='20171201'])>0) & ((Volm_df.loc[Volm_df['YearMonth']=='20171201'])['Volume'].values==0)):
        pred = 0
        pred_volm.append({agency+"-"+sku:int(pred)})
        rt +=1
        #print(rt)

    else:
        m = Prophet().fit(Volm_df)
        future = m.make_future_dataframe(periods=1,freq = 'M')
        fcst = m.predict(future)
        m.plot(fcst)
        print(fcst)
        pred = fcst.iloc[-1,-1]
        if pred <0:
            pred = Volm_df.loc[Volm_df['YearMonth']=='20171201']['Volume'].values
        del m,fcst,future
        pred_volm.append({agency_sku:int(pred)})
        rt +=1
        print(agency_sku,pred)
        
    print('final rt:',rt)
    del Volm_df
    gc.collect()

In [ ]:
# #### predict last data point for each agency - SKU combination
# from tqdm import tqdm
# import warnings
# warnings.filterwarnings("ignore")

# pred_volm = []

# for agency in tqdm(np.unique(hist_volm_upd['Agency'])):
    
#     for sku in np.unique(hist_volm_upd['SKU']):
        
#         pred = 0
#         rt = 0
#         sku = sku
#         agency = agency
        
#         Volm_df = hist_volm_upd.loc[(hist_volm_upd['Agency']==agency) & (hist_volm_upd['SKU']==sku)]
        
#         if len(Volm_df) == 0:
#             pred = 0
#             pred_volm.append({agency+"_"+sku:pred})
#             rt +=1
#             #print(rt)
            
#         elif ((len(Volm_df.loc[Volm_df['YearMonth']=='20171201'])>0) & ((Volm_df.loc[Volm_df['YearMonth']=='20171201'])['Volume'].values==0)):
#             pred = 0
#             pred_volm.append({agency+"_"+sku:pred})
#             rt +=1
#             #print(rt)
            
#         else:
#             m = Prophet().fit(Volm_df)
#             future = m.make_future_dataframe(periods=1,freq = 'M')
#             fcst = m.predict(future)
#             m.plot(fcst)
#             pred = fcst.iloc[-1,-1]
#             if pred <0:
#                 pred = Volm_df.loc[Volm_df['YearMonth']=='20171201']['Volume']
#             del m,fcst,future
#             pred_volm.append({agency+"_"+sku:pred})
#             rt +=1
#             #print(rt)
#         print('final rt:',rt)
#         del Volm_df
        

In [ ]:
pred_volm

In [ ]:
for i in pred_volm:
    print(i)

In [ ]:
#### Build am agency location dataframe
pred_temp_df = pd.DataFrame(np.zeros((len(pred_volm), 2)))
pred_temp_df.columns = ['Agency_SKU',"Volume"]
idx = 0
for i in pred_volm:
    for k,v in i.items():
        print(k,v)
        pred_temp_df.iloc[idx,0] = k
        pred_temp_df.iloc[idx,1] = v
        idx += 1

#pred_temp_df['YearMonth'] = '20180101'
#pred_temp_df = pred_temp_df[['YearMonth','Agency','Avg_Max_Temp']]
pred_temp_df.head()

In [ ]:
temp = []
for agency in tqdm(np.unique(hist_volm_upd['Agency'])):
    for sku in np.unique(hist_volm_upd['SKU']):
        if agency+"-"+sku not in np.unique(pred_temp_df['Agency_SKU']):
            tp = agency+"-"+sku
            temp.append({tp:0})

In [ ]:
temp

In [ ]:
#### Build am agency location dataframe
pred_temp_df1 = pd.DataFrame(np.zeros((len(temp), 2)))
pred_temp_df1.columns = ['Agency_SKU',"Volume"]
idx = 0
for i in temp:
    for k,v in i.items():
        print(k,v)
        pred_temp_df1.iloc[idx,0] = k
        pred_temp_df1.iloc[idx,1] = v
        idx += 1

#pred_temp_df['YearMonth'] = '20180101'
#pred_temp_df = pred_temp_df[['YearMonth','Agency','Avg_Max_Temp']]
pred_temp_df1.head()

In [ ]:
pred_temp_df.head()

In [ ]:
submit = pd.concat([pred_temp_df,pred_temp_df1],axis=0)
submit=pd.DataFrame(submit)
submit.head()

In [ ]:
submit[['Agency','SKU']] = submit['Agency_SKU'].str.split('-',expand=True)

In [ ]:
submit.drop('Agency_SKU', inplace=True)
submit = submit[['Agency','SKU','Volume']]
submit.head()

In [ ]:
submit.to_csv('F:/AV/AbInBev/submit/'+'volume_forecast_v2.csv',index=False)